# Tomography.dev

* Anton Karazeev, you can text me: [anton.karazeev@gmail.com](mailto:anton.karazeev@gmail.com) or [t.me/akarazeev](t.me/akarazeev)

In [77]:
import os
import sys
from math import pi

from IPython.display import Image

from math import pi
import Qconfig
from qiskit import QuantumProgram
from qiskit.tools.visualization import plot_histogram, plot_state
import qiskit.tools.qcvv.tomography as tomo
import pickle

assert sys.version_info >= (3,5), "Only Python 3.5 or greater supported."

In [78]:
# Create your first QuantumProgram object instance.
Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"])

# List of available backends.
Q_program.available_backends()

['ibmqx5',
 'ibmqx4',
 'ibmqx_hpc_qasm_simulator',
 'ibmqx2',
 'ibmqx_qasm_simulator',
 'local_projectq_simulator',
 'local_qasm_simulator',
 'local_clifford_simulator',
 'local_qiskit_simulator',
 'local_unitary_simulator']

In [79]:
n = 1
q_regs = Q_program.create_quantum_register("qr", n)
c_regs = Q_program.create_classical_register("cr", n)

In [80]:
def make_tomography_set(qprogram, circuit_name, qreg, creg):
    # Prepare helper-circuits.
    RxPi = qprogram.create_circuit('tmp_RxPi', [qreg], [creg])
    RxPi.rx(pi, qreg[0])
    RxPi2 = qprogram.create_circuit('tmp_RxPi2', [qreg], [creg])
    RxPi2.rx(pi/2., qreg[0])
    RyPi2 = qprogram.create_circuit('tmp_RyPi2', [qreg], [creg])
    RyPi2.ry(pi/2., qreg[0])
    Meas = qprogram.create_circuit('tmp_Meas', [qreg], [creg])
    Meas.measure(qreg[0], creg[0])
    
    circuit_names = []
    for left in ['=', 'tmp_RxPi', 'tmp_RxPi2', 'tmp_RyPi2']:
        for right in ['=', 'tmp_RxPi2', 'tmp_RyPi2']:
            tmp_name = left + '_' + circuit_name + '_' + right
            tmp_name = tmp_name.replace('tmp_', '')
            circuit_names.append(tmp_name)
            
            tmp_circuit = qprogram.get_circuit(circuit_name)
            if left != '=':
                tmp_circuit = qprogram.get_circuit(left) + tmp_circuit
            if right != '=':
                tmp_circuit = tmp_circuit + qprogram.get_circuit(right)
            
            qprogram.add_circuit(circuit_names[-1], tmp_circuit +
                                 qprogram.get_circuit('tmp_Meas'))

    return circuit_names

In [81]:
# Id on qubit-0 only.
ident = Q_program.create_circuit('Id', [q_regs], [c_regs])
# ident.iden(q_regs[0])

# Rx(Pi) on qubit-0 only.
RxPi = Q_program.create_circuit('RxPi', [q_regs], [c_regs])
RxPi.rx(pi, q_regs[0])

# Rx(Pi/2) on qubit-0 only.
RxPi2 = Q_program.create_circuit('RxPi2', [q_regs], [c_regs])
RxPi2.rx(pi/2., q_regs[0])

# Ry(Pi/2) on qubit-0 only.
RyPi2 = Q_program.create_circuit('RyPi2', [q_regs], [c_regs])
RyPi2.ry(pi/2., q_regs[0])

In [82]:
Q_program.get_circuit_names()

dict_keys(['Id', 'RxPi', 'RxPi2', 'RyPi2'])

In [83]:
tomography = dict()

for name in ['Id', 'RxPi', 'RxPi2', 'RyPi2']:
    tomography[name] = dict(circuits=make_tomography_set(Q_program, name, q_regs, c_regs))

In [90]:
tomography

{'Id': {'circuits': ['=_Id_=',
   '=_Id_RxPi2',
   '=_Id_RyPi2',
   'RxPi_Id_=',
   'RxPi_Id_RxPi2',
   'RxPi_Id_RyPi2',
   'RxPi2_Id_=',
   'RxPi2_Id_RxPi2',
   'RxPi2_Id_RyPi2',
   'RyPi2_Id_=',
   'RyPi2_Id_RxPi2',
   'RyPi2_Id_RyPi2']},
 'RxPi': {'circuits': ['=_RxPi_=',
   '=_RxPi_RxPi2',
   '=_RxPi_RyPi2',
   'RxPi_RxPi_=',
   'RxPi_RxPi_RxPi2',
   'RxPi_RxPi_RyPi2',
   'RxPi2_RxPi_=',
   'RxPi2_RxPi_RxPi2',
   'RxPi2_RxPi_RyPi2',
   'RyPi2_RxPi_=',
   'RyPi2_RxPi_RxPi2',
   'RyPi2_RxPi_RyPi2']},
 'RxPi2': {'circuits': ['=_RxPi2_=',
   '=_RxPi2_RxPi2',
   '=_RxPi2_RyPi2',
   'RxPi_RxPi2_=',
   'RxPi_RxPi2_RxPi2',
   'RxPi_RxPi2_RyPi2',
   'RxPi2_RxPi2_=',
   'RxPi2_RxPi2_RxPi2',
   'RxPi2_RxPi2_RyPi2',
   'RyPi2_RxPi2_=',
   'RyPi2_RxPi2_RxPi2',
   'RyPi2_RxPi2_RyPi2']},
 'RyPi2': {'circuits': ['=_RyPi2_=',
   '=_RyPi2_RxPi2',
   '=_RyPi2_RyPi2',
   'RxPi_RyPi2_=',
   'RxPi_RyPi2_RxPi2',
   'RxPi_RyPi2_RyPi2',
   'RxPi2_RyPi2_=',
   'RxPi2_RyPi2_RxPi2',
   'RxPi2_RyPi2_RyPi2',
  

In [91]:
%%time
tomography_results = dict()

backend = 'local_qasm_simulator'
shots = 1024
for circuit_name in tomography.keys():
    tmp_results = Q_program.execute(tomography[circuit_name]['circuits'], shots=shots, backend=backend)
    print(circuit_name, tmp_results)

    tomography_results[circuit_name] = dict()
    for name in tomography[circuit_name]['circuits']:
        tomography_results[circuit_name][name] = tmp_results.get_data(name)['counts']

Id COMPLETED
RxPi COMPLETED
RxPi2 COMPLETED
RyPi2 COMPLETED
CPU times: user 7.16 s, sys: 384 ms, total: 7.55 s
Wall time: 10.8 s


In [92]:
tomography_results

{'Id': {'=_Id_=': {'0': 1024},
  '=_Id_RxPi2': {'0': 512, '1': 512},
  '=_Id_RyPi2': {'0': 515, '1': 509},
  'RxPi2_Id_=': {'0': 501, '1': 523},
  'RxPi2_Id_RxPi2': {'1': 1024},
  'RxPi2_Id_RyPi2': {'0': 533, '1': 491},
  'RxPi_Id_=': {'1': 1024},
  'RxPi_Id_RxPi2': {'0': 515, '1': 509},
  'RxPi_Id_RyPi2': {'0': 530, '1': 494},
  'RyPi2_Id_=': {'0': 513, '1': 511},
  'RyPi2_Id_RxPi2': {'0': 503, '1': 521},
  'RyPi2_Id_RyPi2': {'1': 1024}},
 'RxPi': {'=_RxPi_=': {'1': 1024},
  '=_RxPi_RxPi2': {'0': 529, '1': 495},
  '=_RxPi_RyPi2': {'0': 487, '1': 537},
  'RxPi2_RxPi_=': {'0': 521, '1': 503},
  'RxPi2_RxPi_RxPi2': {'0': 1024},
  'RxPi2_RxPi_RyPi2': {'0': 487, '1': 537},
  'RxPi_RxPi_=': {'0': 1024},
  'RxPi_RxPi_RxPi2': {'0': 510, '1': 514},
  'RxPi_RxPi_RyPi2': {'0': 517, '1': 507},
  'RyPi2_RxPi_=': {'0': 530, '1': 494},
  'RyPi2_RxPi_RxPi2': {'0': 500, '1': 524},
  'RyPi2_RxPi_RyPi2': {'1': 1024}},
 'RxPi2': {'=_RxPi2_=': {'0': 503, '1': 521},
  '=_RxPi2_RxPi2': {'1': 1024},
  '=_RxP

In [93]:
%%time
tomography_results_real = dict()

backend = 'ibmqx2'
shots = 8192
for circuit_name in tomography.keys():
    tmp_results = Q_program.execute(tomography[circuit_name]['circuits'], shots=shots,
                                    backend=backend, wait=10, timeout=600, max_credits=8)
    print(circuit_name, tmp_results)

    tomography_results_real[circuit_name] = dict()
    for name in tomography[circuit_name]['circuits']:
        tomography_results_real[circuit_name][name] = tmp_results.get_data(name)['counts']

Id COMPLETED
RxPi COMPLETED
RxPi2 COMPLETED
RyPi2 COMPLETED
CPU times: user 10.7 s, sys: 471 ms, total: 11.2 s
Wall time: 30min 19s


In [95]:
!ls

Qconfig.py           data                 main.ipynb
__pycache__          dev_tomography.ipynb tomography.txt


In [96]:
with open('data/tomography_results_real.pkl', 'wb') as file:
    pickle.dump(tomography_results_real, file)

In [97]:
with open('data/tomography_results.pkl', 'wb') as file:
    pickle.dump(tomography_results, file)

In [94]:
tomography_results_real

{'Id': {'=_Id_=': {'00000': 4733, '00001': 3459},
  '=_Id_RxPi2': {'00000': 4498, '00001': 3694},
  '=_Id_RyPi2': {'00000': 4479, '00001': 3713},
  'RxPi2_Id_=': {'00000': 4463, '00001': 3729},
  'RxPi2_Id_RxPi2': {'00000': 4368, '00001': 3824},
  'RxPi2_Id_RyPi2': {'00000': 4532, '00001': 3660},
  'RxPi_Id_=': {'00000': 4241, '00001': 3951},
  'RxPi_Id_RxPi2': {'00000': 4548, '00001': 3644},
  'RxPi_Id_RyPi2': {'00000': 4587, '00001': 3605},
  'RyPi2_Id_=': {'00000': 4560, '00001': 3632},
  'RyPi2_Id_RxPi2': {'00000': 4503, '00001': 3689},
  'RyPi2_Id_RyPi2': {'00000': 4318, '00001': 3874}},
 'RxPi': {'=_RxPi_=': {'00000': 4037, '00001': 4155},
  '=_RxPi_RxPi2': {'00000': 4121, '00001': 4071},
  '=_RxPi_RyPi2': {'00000': 4139, '00001': 4053},
  'RxPi2_RxPi_=': {'00000': 4185, '00001': 4007},
  'RxPi2_RxPi_RxPi2': {'00000': 4200, '00001': 3992},
  'RxPi2_RxPi_RyPi2': {'00000': 4048, '00001': 4144},
  'RxPi_RxPi_=': {'00000': 4161, '00001': 4031},
  'RxPi_RxPi_RxPi2': {'00000': 4123, '0